In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import os
import re

In [2]:
df = pd.read_csv(os.path.join(os.getcwd(), 'data/robocall.csv'))

In [3]:
## Preprocess Time of Issue to consistent format

def preprocessTime(s):
    if isinstance(s, float):
        return s
    s = re.sub(r'[^a-zA-Z\d:]', '', s.lower())
    condition = re.sub(r'[^:]*:[^\D]*', '', s)
    if 2 != len(condition):
        if 'a' in condition:
            s = re.match(r'[^:]*:[^\D]*', s).group(0)+'am'
        else:
            s = re.match(r'[^:]*:[^\D]*', s).group(0)+'pm'
    return s

df['Time of Issue'] = df['Time of Issue'].apply(preprocessTime)

In [4]:
## Dropped all row entries with no provided location...
## Asserts dropped data is insignificant information loss

assert len(df[df['Location (Center point of the Zip Code)'].isnull()])/len(df) < 0.01
df = df.dropna(subset = ['Location (Center point of the Zip Code)'])

In [5]:
## Generate Location dictionary with (zip code, location coordinate) pairs
## Assumes df contains column 'Location (Center point of the Zip Code)'

def getLocations(df, d = dict()):
    splitter = lambda x: tuple(x.split('\n'))
    for entry in set(df['Location (Center point of the Zip Code)'].apply(splitter)):
        if len(entry) == 2:
            k, v = entry
            d[k.split('-')[0].split(' ')[1]] = literal_eval(v)
    return d

locationDict = getLocations(df)

In [6]:
## Preprocess Location and Zip to consistent format

def preprocessLocation(s):
    s = re.sub(r'(\d)\(', r'\1\n(', s)
    return s.split('\n')[0].split(' ')[-1].split('-')[0]

def location(zipCode):
    try:
        return locationDict[zipCode]
    except KeyError:
        return np.nan

df = df.rename(index = str, columns = {'Location (Center point of the Zip Code)': 'Location'})
df['Zip'] = df['Location'].apply(preprocessLocation)
df['Location'] = df['Location'].apply(lambda x: location(preprocessLocation(x)))
df = df.dropna(subset = ['Location'])

In [24]:
df.head()

,Ticket ID,Ticket Created,Date of Issue,Time of Issue,Form,Method,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,City,State,Zip,Location
0,1000296,05/25/2016 11:15:29 AM +0000,05/01/2016,1:00pm,Phone,Internet (VOIP),Robocalls,866-410-0458,Autodialed Live Voice Call,NaN,Plantation,FL,33324,"(26.131328, -80.264664)"
1,1000319,05/25/2016 12:51:35 PM +0000,03/07/2016,12:00pm,Phone,Wired,Telemarketing (including do not call and spoof...,619-840-7262,Live Voice,619-840-7262,San Marcos,CA,92078,"(33.122635, -117.190612)"
2,1000322,05/25/2016 12:56:54 PM +0000,05/24/2016,8:08pm,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,626-691-9090,Live Voice,626-691-9090,Wyckoff,NJ,07481,"(40.981681, -74.189344)"
3,1000323,05/25/2016 01:00:22 PM +0000,03/07/2016,NaN,Phone,Wired,Telemarketing (including do not call and spoof...,877-218-8361,Abandoned Calls,NaN,Hopkington,MA,01748,"(42.224925, -71.537489)"
4,1000326,05/25/2016 01:02:31 PM +0000,05/25/2016,7:24pm,Phone,Internet (VOIP),Telemarketing (including do not call and spoof...,877-705-6767,Abandoned Calls,NaN,Aurora,IL,60504,"(41.745771, -88.239943)"


In [7]:
df

,Ticket ID,Ticket Created,Date of Issue,Time of Issue,Form,Method,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,City,State,Zip,Location
0,1000296,05/25/2016 11:15:29 AM +0000,05/01/2016,1:00pm,Phone,Internet (VOIP),Robocalls,866-410-0458,Autodialed Live Voice Call,NaN,Plantation,FL,33324,"(26.089265, -80.255171)"
1,1000319,05/25/2016 12:51:35 PM +0000,03/07/2016,12:00pm,Phone,Wired,Telemarketing (including do not call and spoof...,619-840-7262,Live Voice,619-840-7262,San Marcos,CA,92078,"(33.124556, -117.21188)"
2,1000322,05/25/2016 12:56:54 PM +0000,05/24/2016,8:08pm,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,626-691-9090,Live Voice,626-691-9090,Wyckoff,NJ,07481,"(41.005131, -74.178312)"
3,1000323,05/25/2016 01:00:22 PM +0000,03/07/2016,NaN,Phone,Wired,Telemarketing (including do not call and spoof...,877-218-8361,Abandoned Calls,NaN,Hopkington,MA,01748,"(42.208013, -71.512306)"
4,1000326,05/25/2016 01:02:31 PM +0000,05/25/2016,7:24pm,Phone,Internet (VOIP),Telemarketing (including do not call and spoof...,877-705-6767,Abandoned Calls,NaN,Aurora,IL,60504,"(41.745771, -88.239943)"
5,1000328,05/25/2016 01:03:17 PM +0000,05/25/2016,7:00am,Phone,Internet (VOIP),Telemarketing (including do not call and spoof...,541-826-3147,Live Voice,NaN,Watertown,NY,13601,"(43.955602, -75.93174)"
6,1000330,05/25/2016 01:10:06 PM +0000,03/14/2016,11:00am,Phone,Wired,Telemarketing (including do not call and spoof...,484-938-8231,Live Voice,484-938-8231,Allentown,PA,18104,"(40.60276, -75.537329)"
7,1000332,05/25/2016 01:16:14 PM +0000,03/14/2016,11:00am,Phone,Wired,Telemarketing (including do not call and spoof...,601-553-0515,NaN,NaN,Meridian,MS,39301,"(32.313297, -88.624084)"
8,1000335,05/25/2016 01:25:05 PM +0000,05/24/2016,2:30pm,Phone,Wired,Robocalls,763-307-5755,Abandoned Calls,NaN,Hot Springs Village,AR,71909,"(34.633828, -92.941276)"
9,1000336,05/25/2016 01:30:02 PM +0000,03/14/2016,2:30pm,Phone,Wired,Telemarketing (including do not call and spoof...,818-666-4016,NaN,NaN,Irvine,CA,92620,"(33.719319, -117.761386)"


In [8]:
df.dropna(subset = ['Date of Issue', 'Time of Issue'])

,Ticket ID,Ticket Created,Date of Issue,Time of Issue,Form,Method,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,City,State,Zip,Location
0,1000296,05/25/2016 11:15:29 AM +0000,05/01/2016,1:00pm,Phone,Internet (VOIP),Robocalls,866-410-0458,Autodialed Live Voice Call,NaN,Plantation,FL,33324,"(26.089265, -80.255171)"
1,1000319,05/25/2016 12:51:35 PM +0000,03/07/2016,12:00pm,Phone,Wired,Telemarketing (including do not call and spoof...,619-840-7262,Live Voice,619-840-7262,San Marcos,CA,92078,"(33.124556, -117.21188)"
2,1000322,05/25/2016 12:56:54 PM +0000,05/24/2016,8:08pm,Phone,Wireless (cell phone/other mobile device),Telemarketing (including do not call and spoof...,626-691-9090,Live Voice,626-691-9090,Wyckoff,NJ,07481,"(41.005131, -74.178312)"
4,1000326,05/25/2016 01:02:31 PM +0000,05/25/2016,7:24pm,Phone,Internet (VOIP),Telemarketing (including do not call and spoof...,877-705-6767,Abandoned Calls,NaN,Aurora,IL,60504,"(41.745771, -88.239943)"
5,1000328,05/25/2016 01:03:17 PM +0000,05/25/2016,7:00am,Phone,Internet (VOIP),Telemarketing (including do not call and spoof...,541-826-3147,Live Voice,NaN,Watertown,NY,13601,"(43.955602, -75.93174)"
6,1000330,05/25/2016 01:10:06 PM +0000,03/14/2016,11:00am,Phone,Wired,Telemarketing (including do not call and spoof...,484-938-8231,Live Voice,484-938-8231,Allentown,PA,18104,"(40.60276, -75.537329)"
7,1000332,05/25/2016 01:16:14 PM +0000,03/14/2016,11:00am,Phone,Wired,Telemarketing (including do not call and spoof...,601-553-0515,NaN,NaN,Meridian,MS,39301,"(32.313297, -88.624084)"
8,1000335,05/25/2016 01:25:05 PM +0000,05/24/2016,2:30pm,Phone,Wired,Robocalls,763-307-5755,Abandoned Calls,NaN,Hot Springs Village,AR,71909,"(34.633828, -92.941276)"
9,1000336,05/25/2016 01:30:02 PM +0000,03/14/2016,2:30pm,Phone,Wired,Telemarketing (including do not call and spoof...,818-666-4016,NaN,NaN,Irvine,CA,92620,"(33.719319, -117.761386)"
10,1000338,05/25/2016 01:30:34 PM +0000,05/24/2016,4:00pm,Phone,Wired,Telemarketing (including do not call and spoof...,214-793-1666,Abandoned Calls,NaN,Hot Springs Village,AR,71909,"(34.633828, -92.941276)"
